# Bayesian Quantum Amplitude Estimation (BAYESQAE) module

The present notebook reviews the **Bayesian Quantum Amplitude Estimation (BAYESQAE)** algorithm.

The **BAYESQAE** algorithm was implemented into the module *bayesian_ae* of the package *AE* of the library *QQuantLib* (**QQuantLib/AE/bayesian_ae.py**). This algorithm is encapsulated in a Python class called `BAYESQAE`.

The present notebook and modules are based on the following reference:

- Alexandra Ramôa and Luis Paulo Santos . Bayesian Quantum Amplitude Estimation. https://arxiv.org/abs/2412.04394 (2024).


In [ ]:
import sys
sys.path.append("../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## QPU
from QQuantLib.qpu.get_qpu import get_qpu
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]
linalg_qpu = get_qpu(my_qpus[1])

In [ ]:
from QQuantLib.DL.data_loading import load_probability

## 1. Oracle generation

Before performing any amplitude estimation, we need to load some data into the quantum circuit. Since this step is auxiliary and intended to illustrate how the algorithm works, we will simply load a discrete probability distribution. In this case, we will use a circuit with $ n = 3 $ qubits, resulting in a total of $ N = 2^n = 8 $ states. The discrete probability distribution we will load is defined as:

$$
p_d = \frac{(0, 1, 2, 3, 4, 5, 6, 7)}{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7}.
$$

In [ ]:
n_qbits = 3
x = np.arange(2**n_qbits)
probability = x/np.sum(x)

Note that this probability distribution is properly normalized. To load this probability distribution into the quantum circuit, we will use the `load_probability` function from the **QQuantLib/DL/data_loading** module. The resulting quantum state can be expressed as:

$$
|\Psi\rangle = \frac{1}{\sqrt{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7}} \left[ \sqrt{0}|0\rangle + \sqrt{1}|1\rangle + \sqrt{2}|2\rangle + \sqrt{3}|3\rangle + \sqrt{4}|4\rangle + \sqrt{5}|5\rangle + \sqrt{6}|6\rangle + \sqrt{7}|7\rangle \right].
$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle --depth 1 --svg

For more information on loading data into the quantum circuit, see the notebook `01_DataLoading_Module_Use.ipynb`

## 2. BAYESQAE algorithm.

### 2.1 The Amplitude Estimation Problem

The problem of amplitude estimation can be stated as follows. Given an oracle operator $\mathcal{A}$:
$$
\mathcal{A}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle + \sqrt{1-a}|\Psi_1\rangle,
$$
where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, the goal is to estimate the value of $\sqrt{a}$. We can associate an angle $\theta$ with $\sqrt{a}$ such that $\sin^2{\theta} = a$, rewriting the problem as:
$$
\mathcal{A}|0\rangle = |\Psi\rangle = \sin(\theta)|\Psi_0\rangle + \cos(\theta)|\Psi_1\rangle. \tag{1}
$$

The foundation of any amplitude estimation algorithm lies in the Grover-like operator $\mathcal{Q}$ derived from the oracle operator $\mathcal{A}$:
$$
\mathcal{Q}(\mathcal{A}) = \mathcal{A} \left(\hat{I} - 2|0\rangle\langle 0|\right) \mathcal{A}^\dagger \left(\hat{I} - 2|\Psi_0\rangle\langle \Psi_0|\right).
$$
This operator acts on the state $|\Psi\rangle$ as follows:
$$
\mathcal{Q}^{m_k}|\Psi\rangle = \mathcal{Q}^{m_k} \mathcal{A} |0\rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle + \cos\left((2m_k+1)\theta\right)|\Psi_1\rangle.
$$

For more information about the Grover operator and the amplitude amplification algorithm, refer to the notebook `02_AmplitudeAmplification_Operators.ipynb`.

### 2.2 BAYESQAE Algorithm Summary

Given an error tolerance $\epsilon$ and a confidence level $\alpha$, the **BAYESQAE** algorithm estimates an interval $(a_l, a_u)$ such that the parameter $a$ in the Amplitude Estimation problem satisfies:
$$
P\big[a \in [a_l, a_u]\big] > 1 - \alpha,
$$
and
$$
\frac{a_u - a_l}{2} \leq \epsilon.
$$

To achieve this estimation, the **BAYESQAE** algorithm combines quantum Grover-like circuits ($\mathcal{Q}^{m_k}|\Psi\rangle$) with a statistical inference framework. The core idea is to start with a prior probability distribution for the value of $a$, and then update it iteratively using the **Bayes rule**, based on the measurement outcomes of $\mathcal{Q}^{m_k}|\Psi\rangle$. When the algorithm terminates, the value of $a$ can be estimated using the final posterior probability distribution.

For the statistical inference, the **BAYESQAE** algorithm operates within the framework of **Sequential Monte Carlo (SMC)** methods. The goal is to use SMC to compute an optimal control for the next **Quantum Amplitude Estimation (QAE)** experiment (i.e., the choice of $m_k$ for the Grover circuit), given the current probability distribution and the results of previous QAE experiments. 

The computation of the optimal control involves minimizing a function called the **Utility of an experiment** over a dynamically evolving range of possible controls (i.e., $m_k$ for the Grover circuits).

### 2.3 Creating an Object from the BAYESQAE Class

We have implemented a Python class called `BAYESQAE` in the **QQuantLib/AE/bayesian_ae** module, which enables the use of the **BAYESQAE** algorithm. To create an instance of the **BAYESQAE** class, the conventions used in the **MLAE** class (from the **QQuantLib/AE/maximum_likelihood_ae** module) should be followed.

#### Mandatory Inputs:
1. `Oracle`: A myQLM AbstractGate or QRoutine object that implements the oracle for constructing the Grover operator.
2. `Target`: The marked state in binary representation, provided as a Python list.
3. `Index`: A list of qubits affected by the Grover operator.

Several additional inputs can be provided as keyword arguments (using a Python dictionary) to configure different parts of the algorithm.

---

#### Quantum Parts Configuration:
The following arguments can be used to configure the quantum components of the algorithm:
- `qpu`: The myQLM solver to be used.
- `mcz_qlm`: A boolean flag indicating whether to use the myQLM multi-controlled Z gate (`True`, default) or a multiplexor implementation (`False`).

---

#### Stopping Condition:
The stopping condition for the **BAYESQAE** algorithm loop is controlled by the following keywords:
- `epsilon` ($\epsilon$): The precision parameter. Ensures that $|a_u - a_l|$ is at most $2\epsilon$.
- `alpha` ($\alpha$): The accuracy parameter. Ensures that the probability of $a$ lying outside the given $\epsilon$ interval is at most $\alpha$ (default: 0.05).
- `max_iterations`: The maximum number of iterations for the **BAYESQAE** algorithm loop. If the algorithm does not satisfy the $\epsilon$ and $\alpha$ requirements within this limit, the loop is truncated.

---

#### Sequential Monte Carlo (SMC) Method Configuration:
The following keywords can be used to configure the SMC method used in the **BAYESQAE** algorithm:
- `particles`: The number of particles used for executing the SMC simulation.
- `threshold`: A float (between 0 and 1) that defines the threshold for triggering resampling of the SMC probability distribution.
- `kernel`: A string specifying the type of perturbation kernel used during resampling. Options are:
  - `LW`: Liu-West perturbation kernel.
  - `Metro`: Metropolis perturbation kernel.
- `alpha_lw`: A float between 0 and 1 for configuring the Liu-West perturbation kernel (used when `kernel="LW"`).
- `c`: A float for configuring the Metropolis perturbation kernel (used when `kernel="Metro"`).

---

#### Dynamic Evolution of Control Domains:
The following keywords can be used to configure the dynamic evolution of the domain controls for minimizing the **Utility of an experiment** function:
- `n_evals`: The number of evaluations for optimizing the expected value of the utility function.
- `k_0`: Together with `n_evals`, defines the upper limit for the initial domain interval for control optimization.
- `R`: when the optimal control is among the top `R` highest possible controls then it can be necessary to enlarge the domain for the controls. An internal counter is increased by one when this happens.
- `T`: If the optimal control is among the top `R` highest possible controls during `T` iterations, the domain interval for control optimization is enlarged (when the internal counter hits `T`)

---

#### Utility Function:


In the original **BAYESQAE** paper, the **Utility of an experiment** function for minimization uses as base function the *variance* (in fact the expected value of the variance). By default, the function `variance_function` from the **QQuantLib/AE/bayesian_ae** module is used. To use a user-defined utility function, the following keyword can be provided:
- `utility_function`: A Python function used for computing the **Utility of an experiment** for the **BAYESQAE** algorithm. The default is the variance function. A custom *utility function* must accept two numpy arrays as inputs:
  - The first array contains the values of the SMC probability distribution.
  - The second array contains the corresponding weights.
  Additionally, the function may accept a `kwargs` argument.

---

#### Shots Configuration:
The **BAYESQAE** algorithm requires performing virtual QAE experiments and updating probabilities using Bayes' rule. The number of shots for these tasks can be configured using the following keywords:
- `shots`: The number of shots used for measuring the quantum part of the algorithm (this is the number of shots used for the complete Grover circuit). Default in the original paper: 1.
- `warm_shots`: The number of shots selected for the warm-up phase (when the number of Grover operator applications is 0). Default in the original paper: 10.
- `bayes_shots`: The number of shots used for Bayesian computations required to update the SMC prior probability distribution to the posterior one. Default in the original paper: 1.
- `control_bayes_shots`: The number of shots used for updating the SMC posterior probabilities during optimization computations.

---

#### Fake Simulation Configuration:
For very low $\epsilon$, the number of Grover operator applications can become prohibitively large, making quantum circuit simulation unfeasible. In such cases, a fake simulation can be used, where **QAE** experiment results are sampled from a binomial distribution with the appropriate probability. The following keywords enable this fake simulation:
- `fake`: A boolean flag. If `False`, the quantum circuit is used for the quantum part of the algorithm. If `True`, the quantum circuit is simulated by sampling from a binomial distribution (the true $\theta$ must be provided).
- `theta_good`: A float representing the true $\theta$ value for fake simulations (i.e., random binomial sampling).

---

#### Other Configuration:
Additional configuration options include:
- `save_smc_prob`: An integer setting the frequency (in number of iterations) for storing the SMC probabilities.
- `print_info`: An integer setting the frequency (in number of iterations) for printing information about the algorithm's evolution.


In [ ]:
#import the class
from QQuantLib.AE.bayesian_ae import BAYESQAE

### 2.4 Toy problem

To show how our class and the algorithm work, we will define the following amplitude estimation problem using the generated oracle operator $\mathcal{A}$ (see Section 2.1)

$$|\Psi\rangle = \mathcal{A}|0\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right] \tag{2}$$

So comparing (2) with (1):

$$\sqrt{a}|\Psi_0\rangle = \sin(\theta)|\Psi_0\rangle = \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle$$

and 

$$\sqrt{1-a}|\Psi_1\rangle = \cos(\theta)|\Psi_1\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

The target state, in this case, is $|1\rangle$. In order to provide a binary representation the `bitfield` function from **QQuantLib.utils.utils** can be used. This has to be passed to the `target` variable as a list. Moreover, we have to provide the list of qubits where we are acting (to the `index` variable), in this case is just the whole register.

In [ ]:
from QQuantLib.utils.utils import bitfield

In [ ]:
index = [i for i in range(oracle.arity)]
# Qubits where the operator should acts
print("Operator should act over the following qubits: {}".format(index))
# Definition of the state
state = 1
# State conversion to binary representation
target = bitfield(state, n_qbits)
print("State: |{}>. Corresponding target:  {}".format(state, target))
# Probability and theta that we want ot estiamte
a_good = probability[state]
theta_good = np.arcsin(np.sqrt(a_good))
print('Real Value of a: ', a_good)
print('theta_good: ', theta_good)

Now that we have the mandatory inputs for instantiating the `BAYESQAE` class (the `oracle`, the `target`, and the `index`), we can proceed to configure the **BAYESQAE** algorithm.

In [ ]:
#Shots
shots = 1
#Stoping condition of the algorithm
epsilon = 1.0e-4
alpha = 0.05

bayes_conf = {
    # Quantum parts related
    'qpu': linalg_qpu,
    'mcz_qlm': True,
    # Stoping condition
    "epsilon" : epsilon,
    "alpha": alpha,
    "max_iterations": 1000,
    # Configuration of the SMC method
    "particles": 2000,      
    "threshold": 0.5,    
    "kernel" : "LW", #"Metro", #LW
    "alpha_lw":0.9, # Liu-West kernel configuration
    "c" : 2.38,   # Metropoli kernel configuration 
    # Dinamyc evolution of the domain controls
    "k_0":2, 
    "T" : 3,
    "R" : 3,    
    "n_evals" : 50,    
    # Shots configuration
    "shots" : shots,
    "warm_shots":10,
    "bayes_shots": shots,
    "control_bayes_shots" : shots,
    # Fake configuration
    "fake": False,
    "theta_good": theta_good,    
    # Other configuration
    "save_smc_prob" : 1,
    "print_info" : 1,    
}

In [ ]:
#instantiate the Class
bayes = BAYESQAE(
    oracle,
    target=target,
    index=index,
    **bayes_conf
)

### 2.5 BAYESQAE Workflow

In this section, we explain the workflow of the **BAYESQAE** algorithm using the `BAYESQAE` class and other implemented functions from the **QQuantLib/AE/bayesian_ae** module.

#### 2.5.1 Building the SMC Prior Probability Distribution

The first step in the **BAYESQAE** algorithm is to construct the prior probability distribution. In **Sequential Monte Carlo (SMC)** methods, the probability distribution is represented by two arrays:
- One array contains the possible values of the parameter to be estimated: $\{\theta^{prior}_i\}$.
- The second array contains the corresponding weights: $\{W^{prior}_i\}$, which depend on the desired prior probability distribution.

The weights **must** be normalized such that:
$$
\sum_i W^{prior}_i = 1.
$$

The index $i$ runs from 1 to the desired number of particles, which is an input parameter of the algorithm (`particles`).

Typically, a uniform prior probability distribution is used unless prior knowledge about the parameter suggests otherwise. In our implementation, we always start with a uniform probability distribution for the $\theta$ value to estimate (recall that $\sin^2{\theta} = a$) over the interval $\left[0, \frac{\pi}{2}\right]$.

The following cell builds the **SMC** prior probability distribution.

In [ ]:
# We use the particles attribute of the class
theta_prior = np.random.uniform(0.0, 0.5 * np.pi, bayes.particles)
# All the particles will have associated the same weights
weights_prior = np.ones(len(theta_prior)) / len(theta_prior)

#### 2.5.2 Warm-Up Measurement

The second step in the **BAYESQAE** algorithm is the warm-up phase. During this phase, we measure the bare oracle operator $\mathcal{A}$ to obtain initial results. This is achieved using the `quantum_measure_step` method of the **BAYESQAE** object.

To perform this measurement, we need to specify:
- The number of controls, $m_k$, which should be set to 0 during the warm-up phase.
- The number of shots for measuring the circuit, which is determined by the `warm_shots` input parameter.

The output of this step is the probability of measuring the *target* state. Additionally, the method provides the quantum circuit used for the measurement.

In [ ]:
m_0 = 0
warm_outcome, routine = bayes.quantum_measure_step(m_0, bayes.warm_shots)
print("Probability measured for warm up phase: {}".format(warm_outcome))
# Transform to measures
warm_outcome = round(warm_outcome * bayes.warm_shots)
print("Measured events for warm up phase: {}".format(warm_outcome))

# For storing all QAE experiments
control_list = []
shots_list = []
outcome_list = []

control_list.append(m_0)
shots_list.append(bayes.warm_shots)
outcome_list.append(warm_outcome)

#### 2.5.3 Updating the SMC Posterior Probability with Warm-Up Results

After obtaining the results from the warm-up **QAE** experiment, we update the prior probability distribution to the posterior probability using Bayes' theorem. This can be achieved using the `bayesian_update` function from the **QQuantLib/AE/bayesian_ae** module.

This function accepts the following arguments:
- `thetas`: The values of the SMC prior distribution ($\{\theta^{prior}_i\}$).
- `weights`: The weights of the SMC prior distribution ($\{W^{prior}_i\}$).
- `m_k`: A list of all the controls used in the **QAE** experiments up to this point.
- `n_k`: A list of all the shots used in the **QAE** experiments up to this point.
- `o_k`: A list of all the outcomes obtained from the **QAE** experiments up to this point.
- `resample`: A boolean flag indicating whether resampling should be performed.
- `kwargs`: Additional arguments for configuring the resampling kernels.

The workflow of the `bayesian_update` function is as follows:

1. **Compute New Weights Using the Last QAE Experiment**:
   - For each possible value of $\theta$ in the input prior distribution, compute the likelihood based on the result of the last **QAE** experiment:
     $$
     L(\theta | h_k; m_k, n_k) = \sin^2\left((2m_k+1)\theta\right)^{h_k} \cos^2\left((2m_k+1)\theta\right)^{n_k-h_k}.
     $$
   - Update the new weights using the computed likelihoods:
     $$
     w^{posterior}_i = L(\theta^{prior}_i | h_k; m_k, n_k) \cdot W^{prior}_i.
     $$
   - Normalize the new weights:
     $$
     W^{posterior}_i = \frac{w^{posterior}_i}{\sum w^{posterior}_i}.
     $$

2. **Construct the Posterior Distribution**:
   - The posterior distribution is defined by the prior values ($\{\theta^{posterior}_i\} = \{\theta^{prior}_i\}$) and the updated weights ($W^{posterior}_i$).
   - A potential issue arises when many weights are close to zero, making the probability distribution less useful for subsequent steps. To address this, the Bayesian update procedure is typically followed by a resampling protocol.

3. **Resampling Protocol**:
   - Resampling occurs if the **effective sample size (ESS)** of the weights falls below a specified `threshold` multiplied by the number of `particles`. The ESS is calculated as (the `ess` function from **QQuantLib.AE.bayesian_ae** executes this computation):
     $$
     ESS = \frac{\left(\sum_i W^{posterior}_i\right)^2}{\sum_i {W^{posterior}_i}^2}.
     $$

4. **Resampling Procedure**:
   - New $\theta^{posterior}$ values are selected from the prior $\theta^{prior}$ based on the probabilities given by their associated posterior weights ($W^{posterior}_i$).
   - This process eliminates $\theta$ values with very low weights and duplicates those with higher weights.

5. **Apply Perturbation Kernel**:
   - A perturbation kernel is applied to the resampled $\theta^{posterior}$ values to introduce slight variations around the original values. Two kernel methods are supported:
     - **Liu-West Kernel**: Set the `kernel` attribute to `"LW"` and configure the `alpha_lw` parameter appropriately (a value between 0 and 1).
     - **Metropolis Kernel**: Set the `kernel` attribute to `"Metro"` and configure the `c` parameter appropriately (a value of 2.38 often works well).

6. **Reset Weights After Resampling**:
   - If resampling takes place, the new weights are reset to a uniform distribution:
     $$
     W^{posterior}_i = \frac{1}{\text{particles}}.
     $$

This process ensures that the posterior distribution remains robust and well-distributed for subsequent iterations of the algorithm.


In [ ]:
from QQuantLib.AE.bayesian_ae import bayesian_update, ess

In [ ]:
# first update the SMC posterior without allowing resampling

theta_posterior, weights_posterior = bayesian_update(
    theta_prior, weights_prior, # prior SMC 
    control_list, shots_list, outcome_list, # QAE experiment until the moment
    resample=False, # Not resampliing
)


In [ ]:
plt.plot(theta_prior, weights_prior, 'o')
plt.plot(theta_posterior, weights_posterior, 'o')
plt.legend(["SMC prior", "SMC posterior"])
plt.xlabel(r"$\theta$")
plt.ylabel(r"Weights($W$)")

As can be observed, there are many $\theta$ values with weights close to zero. The **effective sample size (ESS)** can be used to determine whether resampling is necessary. 


In [ ]:
print("Resampling needed: {}".format(ess(weights_posterior) < bayes.threshold * bayes.particles))

If resampling is determined to be necessary, we can specify which kernel should be used for perturbing the resampled particles. 

In [ ]:
# Liu-West Kernel
lw_conf = {"kernel" : "LW", "alpha_lw" : bayes.alpha_lw}
theta_posterior_lw, weights_posterior_lw = bayesian_update(
    theta_prior, weights_prior, # prior SMC 
    [m_0], [bayes.warm_shots], [warm_outcome], # QAE experiment until the moment
    resample=True, # Not resampling
    **lw_conf
)
#Metropoli Kernel
metro_conf = {"kernel" : "Metro", "c" : bayes.c}
theta_posterior_metro, weights_posterior_metro = bayesian_update(
    theta_prior, weights_prior, # prior SMC 
    [m_0], [bayes.warm_shots], [warm_outcome], # QAE experiment until the moment
    resample=True, # Not resampling
    **metro_conf
)

Now, we can visualize how the different kernels impact the SMC posterior probability distribution. By plotting the results, we can observe the effects of the perturbation introduced by each kernel on the resampled particles. This allows us to compare the behavior of the **Liu-West** and **Metropolis** kernels and understand their influence on the final distribution.

In [ ]:
plt.hist(theta_prior)
plt.hist(theta_posterior_lw, alpha=0.8)
plt.hist(theta_posterior_metro, alpha=0.8)
plt.legend(["Prior", "Posterior LW", "Posterior Metro"])

It is important to remember that when resampling is performed, the weights of the particles are reinitialized to have equal probabilities. This ensures that all resampled particles contribute equally to the subsequent steps of the algorithm, maintaining a balanced representation of the probability distribution.

In [ ]:
plt.plot(theta_posterior_lw, weights_posterior_lw, 'o')
plt.plot(theta_posterior_metro, weights_posterior_metro, 'o')
plt.legend(["LW", "Metro"])
plt.xlabel(r"$\theta$")
plt.ylabel(r"Weights($W$)")

Now that we have the SMC posterior distribution ($\{\theta^{posterior}_i\}$ and $\{W^{posterior}_i\}$), we can estimate the desired parameter ($a$ or $\theta$) by computing the **expected value of the mean** under the SMC posterior distribution.

In the **SMC** framework, the expected value of any function $f$ under an SMC distribution is computed as follows:
$$
E_{\theta, W}\left[f(x)\right] = \sum_i f(\theta^{posterior}_i) W^{posterior}_i
$$

Thus, the desired parameter estimation, $a$, can be computed as:

$$
\hat{a} = \sum_i \sin^2(\theta^{posterior}_i) W_i^{posterior}
$$

In [ ]:
# We use Metro Kernel results
theta_posterior, weights_posterior = theta_posterior_metro, weights_posterior_metro
# Computing the estimations
a_estimation = (np.sin(theta_posterior) ** 2) @ weights_posterior
print("Estimation of the a: {}. Real a value: {}".format(a_estimation, a_good))

The confidence interval for a given confidence level $\alpha$ can be computed using the SMC posterior distribution. The confidence interval $\left[\theta^{\alpha}_l, \theta^{\alpha}_u\right]$ is defined such that:

$$
P_{\theta, W}[\theta > \theta^{\alpha}_l \; \text{and} \; \theta < \theta^{\alpha}_u] \geq 1.0 - \alpha
$$

This confidence interval serves as the **stopping condition** for the algorithm. Specifically, we require the size of the confidence interval to be smaller than an input threshold $\epsilon$ (specified via the `epsilon` keyword):

$$
\left|a^{\alpha}_u - a^{\alpha}_l\right| < \frac{\epsilon}{2}
$$

Here, $a^{\alpha}_{u, l}$ is derived from the bounds of the confidence interval for $\theta$ as follows:
$$
a^{\alpha}_{u, l} = \sin^2{\theta^{\alpha}_{u, l}}
$$

The confidence interval can be computed using the `confidence_intervals` function from the **QQuantLib.AE.bayesian_ae** module.

In [ ]:
from QQuantLib.AE.bayesian_ae import confidence_intervals

In [ ]:
print("Cofidence level: {}".format(bayes.alpha))
theta_l, theta_u = confidence_intervals(theta_posterior, weights_posterior, bayes.alpha)
a_l = np.sin(theta_l) ** 2
a_u = np.sin(theta_u) ** 2
print("confidence inteval: ({}, {})".format(a_l, a_u))

In [ ]:
plt.hist(np.sin(theta_posterior) ** 2)
plt.axvline(a_l, c="r")
plt.axvline(a_u, c="r")

In [ ]:
lenght_interval = np.abs(a_u-a_l)
print("The length of the confidence interval for confidence level {} is: {}".format(
    bayes.alpha, lenght_interval,
))
print("The desired epsilon is: {}".format(bayes.epsilon))
print("Stop condition: {}".format(lenght_interval < bayes.epsilon))

#### 2.5.4 Improving Estimation Using Grover Circuits

The **BAYESQAE** algorithm now begins to iteratively use Grover circuits to refine the estimation. In each iteration, the algorithm computes an optimal control $m^{*}_k$ based on the SMC posterior distribution from the previous iteration (i.e., the posterior distribution from the previous iteration becomes the prior distribution for the current iteration). The algorithm then performs a **Quantum Amplitude Estimation (QAE)** experiment with $m^{*}_k$ and updates the SMC posterior probability using the results. These iterations continue until the stopping condition is satisfied.

The workflow for iteration $k$ is as follows:

1. **Convert Old Posterior into New Priors**:
   - Set $\theta^{prior}_{i,k} \leftarrow \theta^{posterior}_{i, k-1}$.
   - Set $W^{prior}_{i, k} \leftarrow W^{posterior}_{i, k-1}$.

2. **Compute Optimal Control**:
   - Compute the optimal control $m^*_k$ using $\theta^{prior}_{i,k}$, $W^{prior}_{i,k}$, and the results of all **QAE** experiments performed up to iteration $k$.

3. **Execute QAE Experiment**:
   - Perform the **QAE** experiment: $\mathcal{Q}^{m^*_k}|\Psi\rangle$, using $n_k$ shots, and obtain the number of good states measured: $o_k$.

4. **Update SMC Posterior Probability**:
   - Use the `bayesian_update` function to update the SMC posterior probability:
     $$
     \theta^{prior}_{i,k}, W^{prior}_{i,k}, m^*_k, n_k, o_k \rightarrow \theta^{posterior}_{i, k}, W^{posterior}_{i, k}.
     $$

5. **Check Stopping Condition**:
   - Evaluate whether the stopping condition is satisfied. If not, return to step 1; otherwise, terminate the algorithm.

This iterative process ensures that the estimation progressively improves until the desired precision is achieved.

#### 2.5.5 Computing the Optimal Control

From the workflow presented in the previous section, the only step that remains to be explained is the second one: **Compute Optimal Control**.

Before diving into this step, we need an important ingredient: a function that computes the **Utility of an experiment** for a given control $m_k$ and an input SMC probability distribution.

---

##### Utility of an Experiment

The **Utility of an experiment** is computed using the following formula:

$$
U^{f}(m_k) = \sum_{D} E_{P(\theta)}[P(D; m_k)] \cdot E_{P(\theta|D, m_k)}[f(\theta, D; m_k)]
$$

Where:
- $f$: An input utility function defined by the user (keyword argument: `utility_function`). In the original paper, this is typically the variance
- $E_{P(\theta)}[P(D; m_k)]$: The probability of obtaining the outcome $D$ for a **QAE** experiment with control $m_k$. This is computed using Bayes' rule:
  - $E_{P(\theta)}[P(D; m_k)] = \int L(\theta|D; m_k) P(\theta) d\theta$
  - $L(\theta|D; m_k)$: The likelihood of obtaining the outcome $D$ for a **QAE** experiment with control $m_k$.
  - If the input SMC probability is given by $\{\theta^{prior}_i, W^{prior}_i\}$, this computation simplifies to:
    $$
    E_{P(\theta)}[P(D; m_k)] = \sum_i L(\theta^{prior}_i|D; m_k) W^{prior}_i
    $$

- $E_{P(\theta|D, m_k)}[f(\theta, D; m_k)]$: The expected value of the *utility function* for an outcome $D$ of a **QAE** experiment with control $m_k$.
  - If the input SMC probability is given by $\{\theta^{prior}_i, W^{prior}_i\}$, the computation involves performing a hypothetical Bayesian update to compute the corresponding posterior distribution $\{\theta^{posterior}_i, W^{posterior}_i\}$ using the possible outcome $D$ from a **QAE** experiment with control $m_k$.
     $$
     \theta^{prior}_{i}, W^{prior}_{i}, m_k, D \rightarrow \theta^{posterior}_{i}, W^{posterior}_{i}.
     $$  
  - The computation is done as follows:
    $$
    E_{P(\theta|D, m_k)}[f(\theta, D; m_k)] = \sum_i f(\theta^{posterior}_i) \cdot W^{posterior}_i
    $$

---

To compute the **utility of the experiment** ($U^{f}(m_k)$), the `average_expectation` function from the **QQuantLib.AE.bayesian_ae** module is used. This function accepts the following inputs:
- `thetas`: The values of the SMC prior distribution ($\{\theta^{prior}_i\}$).
- `weights`: The weights of the SMC prior distribution ($\{W^{prior}_i\}$).
- `m_k`: A possible input control $m_k$.
- `n_k`: The number of shots $n_k$ for a hypothetical **QAE** experiment.
- `kwargs`: Other keyword arguments (It must have the `utility_function` one, that pass a python function)



In [ ]:
from QQuantLib.AE.bayesian_ae import average_expectation, variance_function

In [ ]:
m_k = 2
n_k = 1
conf_u = {"utility_function": variance_function}
utility = average_expectation(
    theta_posterior, weights_posterior, m_k, n_k, **conf_u
)
print("For m_k:{} and n_k:{} the utility will be: {}".format(
    m_k, n_k, utility
))

In [ ]:
m_k = 10
n_k = 1
conf_u = {"utility_function": variance_function}
utility = average_expectation(
    theta_posterior, weights_posterior, m_k, n_k, **conf_u
)
print("For m_k:{} and n_k:{} the utility will be: {}".format(
    m_k, n_k, utility
))

##### Control Optimization Using the **Utility**

The **Utility of an experiment** is used to compute the **Optimal Control** for the next iteration. This is achieved by minimizing the **Utility of an experiment** over a predefined control domain. The `optimize_control` method of the `BAYESQAE` class performs this minimization and outputs the optimal control $m^*_k$.

Initially, the predefined control domain is initialized to the interval $\left[m_{min}, m_{max}\right]$, where:
- $m_{min} = 0$
- $m_{max} = k_0 \cdot n_{evals}$

Here, $k_0$ and $n_{evals}$ are hyperparameters of the algorithm. The **Utility of an experiment** is computed for $n_{evals}$ possible controls within this interval, and the $m^*_k$ that minimizes it is selected.

However, as the algorithm progresses through multiple iterations, the initial domain interval may no longer provide a suitable minimum for the **Utility of an experiment**. In such cases, it becomes necessary to enlarge the control domain interval. This enlargement is controlled by two hyperparameters: $R$ and $T$.

- Each time the selected $m^*_k$ falls within the top $R$ tested controls, an internal counter is incremented by one.
- When the counter reaches the threshold $T$, the control domain interval is enlarged by setting:
  - $m_{min} \leftarrow m_{max}$
  - $m_{max} \leftarrow 2 \cdot m_{max}$
- The internal counter is reset to zero, and the iteration process continues using the new control domain interval for computing $m^*_k$.

---

### Inputs of the `optimize_control` Method:
- `thetas`: The values of the SMC prior distribution ($\{\theta^{prior}_i\}$).
- `weights`: The weights of the SMC prior distribution ($\{W^{prior}_i\}$).
- `control_bayes_shots`: The number of shots used for the hypothetical **QAE** experiment required to compute the minimum of the **Utility** of an experiment. In the original paper, this is set to 1.
- `kwargs`: Keyword arguments for configuring the `optimize_control` method:
  - `n_evals`: Number of evaluations ($n_{evals}$) for testing controls within the domain.
  - `k_0`: Scaling factor ($k_0$) for initializing the control domain.
  - `R`: Threshold for determining when the selected $m^*_k$ is among the top tested controls.
  - `T`: Counter threshold for triggering the enlargement of the control domain interval.
  - `utility_function`: Base function for computing the **Utility of an experiment**.

In [ ]:
# We use the last step SMC posterior probability for computing new 

# Configure the optimal control
conf = {
    "R": bayes.R, "T": bayes.T, "k_0": bayes.k_0, "n_evals": bayes.n_evals,
    "utility_function": bayes.utility_function
}
# Control for next iteration
optimal_control = bayes.optimize_control(
    theta_posterior, weights_posterior, 
    bayes.control_bayes_shots, **conf
)
print("The optimal control for the following iteration is: {}".format(optimal_control))

The attributes `m_min` and `m_max` of the `BAYESQAE` object store the interval for the domain control ($m_{min}$ and $m_{max}$).


In [ ]:
print("Domain interval control: [{}, {}]".format(bayes.m_min, bayes.m_max))

Now, we can iteratively execute the workflow presented in Section 2.5.5 until the desired `epsilon` is achieved. 

The following cell can be run multiple times until the stopping condition is satisfied. Each execution refines the estimation by updating the SMC posterior distribution and computing a new optimal control $m_k^*$, bringing the algorithm closer to the desired precision.

In [ ]:
# Steps 1 and 2 of section 2.5.5
optimal_control = bayes.optimize_control(
    theta_posterior, weights_posterior, 
    bayes.control_bayes_shots, **conf
)
print("The optimal control for the following iteration is: {}".format(optimal_control))
print("Domain interval control: [{}, {}]".format(bayes.m_min, bayes.m_max))
# Step 3 of section 2.5.5
p_m, routine = bayes.quantum_measure_step(
    optimal_control, bayes.shots
)

control_list.append(optimal_control)
shots_list.append(bayes.shots)
outcome_list.append(round(p_m * bayes.shots))
# Step 4 of section 2.5.5
theta_posterior, weights_posterior = bayesian_update(
    theta_posterior, weights_posterior,
    control_list, shots_list, outcome_list, **metro_conf

)    
# Step 5 of section 2.5.5
a_estimation = (np.sin(theta_posterior) ** 2) @ weights_posterior
theta_l, theta_u = confidence_intervals(theta_posterior, weights_posterior, bayes.alpha)
a_l = np.sin(theta_l) ** 2
a_u = np.sin(theta_u) ** 2
print("Estimation of the a: {}. \n\t confidence inteval: ({}, {})".format(
    a_estimation, a_l, a_u)
)

stop_condition = (a_u-a_l) < 2 * bayes.epsilon
print("Stop condition: {}".format(stop_condition))

In [ ]:
print("Absolute Error: {}".format(np.abs(a_estimation-a_good)))

We can plot the final SMC posterior probability:

In [ ]:
plt.hist(np.sin(theta_posterior)**2)

## 3. Complete Execution of BAYESQAE

Section 2 provided a detailed explanation of the **BAYESQAE** algorithm, with an emphasis on the inner workings and the use of various functions and methods from **QQuantLib.AE.bayesian_ae** for pedagogical purposes. The `BAYESQAE` class encapsulates the entire workflow described in Section 2, abstracting away the complexities and providing a user-friendly interface.

For a complete **BAYESQAE** execution, users are expected to interact with only the following methods:
- `bayesqae`
- `run`

The following sub sections explain them.

### 3.1 The `bayesqae` Method

To execute the complete **BAYESQAE** algorithm, the `bayesqae` method is used. The user must first initialize the class by providing the minimum mandatory inputs (`oracle`, `target`, and `index`). It is recomended too provided info about the quantum part of the algorithm (keyword `qpu`). Once the class is initialized, the `bayesqae` method can be executed by supplying a Python dictionary containing all the necessary configuration parameters.

This approach allows users to provide a comprehensive set of inputs in a single step, ensuring that the algorithm is fully configured before execution. The method handles the entire workflow internally, abstracting away the complexities of the underlying processes.

This method populates the following attributes:

- `mean_a`: evolution of the expected value of the mean throughout the entire algorithm's execution.
- `lower_a`: evolution of the lower value for confidence interval throughout the entire algorithm's execution.
- `upper_a`: evolution of the upper value for confidence interval throughout the entire algorithm's execution.
- `control_list`: evolution of the different controls used for the **QAE** experiments performed throughout the entire algorithm's execution.
- `shots_list`: number of shots used for the **QAE** experiments performed throughout the entire algorithm's execution.
- `outcome_list`: evolution of the different outcomes from the **QAE** experiments performed throughout the entire algorithm's execution.
- `pdf_theta`: evolution of the probability distribution values throughout the entire algorithm's execution.
- `pdf_weights`: evolution of the probability distribution weights throughout the entire algorithm's execution.

In [ ]:
#instantiate the Class

# It is recomeded to initialize the quantum part when the class is instantiated
qpu_conf = {
    # Quantum parts related
    'qpu': linalg_qpu,
}

bayes = BAYESQAE(
    oracle,
    target=target,
    index=index,
    **qpu_conf
)

In [ ]:
#Shots
shots = 1
#Stoping condition of the algorithm
epsilon = 1.0e-3
alpha = 0.05

bayes_conf = {
    # Stoping condition
    "epsilon" : epsilon,
    "alpha": alpha,
    "max_iterations": 1000,
    # Configuration of the SMC method
    "particles": 2000,      
    "threshold": 0.5,    
    "kernel" : "LW", #"Metro", #LW
    "alpha_lw":0.9, # Liu-West kernel configuration
    "c" : 2.38,   # Metropoli kernel configuration 
    # Dinamyc evolution of the domain controls
    "k_0":2, 
    "T" : 3,
    "R" : 3,    
    "n_evals" : 50,    
    # Shots configuration
    "shots" : shots,
    "warm_shots":10,
    "bayes_shots": shots,
    "control_bayes_shots" : shots,
    # Fake configuration
    "fake": False,
    "theta_good": theta_good,    
    # Other configuration
    "save_smc_prob" : 1,
    "print_info" : 1,    
}

In [ ]:
a_mean, a_l, a_u = bayes.bayesqae(**bayes_conf)

In [ ]:
print("Estimation: {}. Confidence intervals: [{},{}]".format(
    a_mean, a_l, a_u
))
print("Error: {}".format(abs(a_mean-a_good)))


The class attributes `mean_a`, `lower_a`, and `upper_a` store the evolution of the expected value of the mean and the confidence intervals throughout the entire algorithm's execution. These attributes provide insight into how the estimates for $a$ improve iteratively as the **BAYESQAE** algorithm progresses.

In [ ]:
plt.plot(bayes.lower_a, '-o')
plt.plot(bayes.mean_a, '-o')
plt.plot(bayes.upper_a, '-o')
plt.legend(["a_l", "a_mean", "a_u"])
plt.ylabel(r"a")
plt.xlabel("iteration")

Additionally, the attributes `outcome_list`, `control_list`, and `shots_list` store the details of the different **QAE** experiments performed during the algorithm's execution. These attributes provide a record of the outcomes, controls, and shot configurations used in each experiment, allowing for a detailed analysis of the algorithm's behavior and results.

In [ ]:
bayes.outcome_list

In [ ]:
bayes.control_list

The class attributes `pdf_theta` and `pdf_weights` are Pandas DataFrames that store the SMC probability distribution (values and weights, respectively) at various stages of the algorithm's iterations. The frequency with which these distributions are saved is controlled by the keyword argument `save_smc_prob`. This allows users to analyze the evolution of the SMC probability distribution over time, providing insights into how the algorithm refines its estimates.

In [ ]:
bayes.pdf_theta 

In [ ]:
bayes.pdf_weights

The following cell generates an animation illustrating the evolution of the SMC probability distribution across different iterations of the algorithm. This visualization provides insight into how the distribution refines and converges as the **BAYESQAE** process progresses.

In [ ]:
pdf_theta = bayes.pdf_theta
%matplotlib notebook
from matplotlib.animation import FuncAnimation
fig, ax = plt.subplots()
def update(frame):
    
    # Limpiar el eje para evitar superposición
    ax.cla()
    
    # Seleccionar la columna correspondiente al cuadro actual
    column_name = pdf_theta.columns[frame]
    data = pdf_theta[column_name]
    
    # Dibujar el histograma
    #ax.hist(pdf_theta["prior"], bins=20, edgecolor='black')
    ax.hist(data, bins=20, color='skyblue', edgecolor='black')
    
    # Añadir título y etiquetas
    ax.set_title(f'Histograma of {column_name}')
    #ax.set_xlabel('Valores')
    #ax.set_ylabel('Frecuencia')
    
    # Devolver el eje
    return ax
# Número de frames igual al número de columnas
num_frames = len(pdf_theta.columns)
ani = FuncAnimation(fig, update, frames=num_frames, interval=1000, repeat=True)
plt.show()

In [ ]:
%matplotlib inline

We can update the input dictionary and perform the algorithm again

In [ ]:
# Changing the shots
bayes_conf.update({
    "shots" : 1000,
    "warm_shots":100,
    "bayes_shots": 100,
    "control_bayes_shots" : 1,
    "epsilon" : 1.0e-4,        
})
a_mean, a_l, a_u = bayes.bayesqae(**bayes_conf)

In [ ]:
# Here we use fake binomial sampling for simulating quantum step
bayes_conf.update({
    "shots" : 1,
    "warm_shots":1,
    "bayes_shots": 1,
    "control_bayes_shots" : 1,
    "epsilon" : 1.0e-10,   
    "save_smc_prob" : 10,
    "print_info" : 10,  
    "fake":True
})
a_mean, a_l, a_u = bayes.bayesqae(**bayes_conf)

In [ ]:
%matplotlib inline
plt.plot(bayes.lower_a, '-o')
plt.plot(bayes.mean_a, '-o')
plt.plot(bayes.upper_a, '-o')
plt.legend(["a_l", "a_mean", "a_u"])
plt.ylabel(r"a")
plt.xlabel("iteration")
plt.xscale("log")
plt.yscale("log")

### 3.1 The `run` Method

The `run` method executes the **BAYESQAE** algorithm directly using the configuration provided during the instantiation of the `BAYESQAE` class or the default configuration if none is specified. This method automates the entire workflow, handling all intermediate steps and iterations until the stopping condition is satisfied.

Upon completion, the `run` method populates the following attributes:

- `ae` :  final exepected mean value of the parameter $a$ (computed using the final posterior distribution)
- `ae_l` :  final lower value of the confidence interval for aprameter $a$ (computed using the final posterior distribution)
- `ae_u`:  final upper value of the confidence interval for aprameter $a$ (computed using the final posterior distribution)
- `schedule_pdf`: A Pandas DataFrame that summarizes the **QAE** experiments and their results performed during the execution of the algorithm.
- `oracle_calls`: The total number of oracle calls made during the complete execution of the algorithm, accounting for the number of shots used in each experiment.
- `max_oracle_depth`: The maximum number of applications of the oracle during the execution of the algorithm (not accounting for shots). This indicates the depth of the largest quantum circuit used.
- `pdf_estimation`: A Pandas DataFrame that tracks the evolution of the mean estimation and the upper and lower bounds of the confidence intervals throughout the algorithm's execution.
- `run_time`: the elapsed time of the complete **BAYESQAE** algorithm
- `quantum_time`: total time the algorithm expends in the pure quantum part.

These attributes provide valuable insights into the performance and behavior of the algorithm, enabling users to analyze its efficiency and accuracy.

In [ ]:
#Shots
shots = 1
#Stoping condition of the algorithm
epsilon = 1.0e-4
alpha = 0.05

bayes_conf = {
    'qpu': linalg_qpu,    
    # Stoping condition
    "epsilon" : epsilon,
    "alpha": alpha,
    "max_iterations": 1000,
    # Configuration of the SMC method
    "particles": 2000,      
    "threshold": 0.5,    
    "kernel" : "LW", #"Metro", #LW
    "alpha_lw":0.9, # Liu-West kernel configuration
    "c" : 2.38,   # Metropoli kernel configuration 
    # Dinamyc evolution of the domain controls
    "k_0":2, 
    "T" : 3,
    "R" : 3,    
    "n_evals" : 50,    
    # Shots configuration
    "shots" : shots,
    "warm_shots":10,
    "bayes_shots": shots,
    "control_bayes_shots" : 1,
    # Fake configuration
    "fake": False,
    "theta_good": theta_good,    
    # Other configuration
    "save_smc_prob" : 10,
    "print_info" : 10,    
}
bayes = BAYESQAE(
    oracle,
    target=target,
    index=index,
    **bayes_conf
)
a_estimated = bayes.run()

In [ ]:
%matplotlib inline
plt.plot(bayes.lower_a, '-o')
plt.plot(bayes.mean_a, '-o')
plt.plot(bayes.upper_a, '-o')
plt.legend(["a_l", "a_mean", "a_u"])
plt.ylabel(r"a")
plt.xlabel("iteration")
plt.xscale("log")
plt.yscale("log")

In [ ]:
print('a_estimated: ', a_estimated)
print('Real Value of a: ', a_good)
print('Bounds for a: [bayes.ae_l, bayes.ae_u] = [{}, {}]'.format(bayes.ae_l, bayes.ae_u))
print('Estimated a: bayes.ae = ', bayes.ae)

In [ ]:
print(' a_real-iqbayesae.ae: ', abs(bayes.ae-a_good))
print('Epsilon required: ', bayes.epsilon)
print('Bayes lenght of confidence intervals ', 0.5 * (bayes.ae_u-bayes.ae_l))

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls was: {}".format(bayes.oracle_calls))
#Total number of oracle calls
print("The maximum depth oracle circuit was: {}".format(bayes.max_oracle_depth))

print("Elapsed time for the run method: ", bayes.run_time)
print("Time of the quantum parts: ", bayes.quantum_time)

In [ ]:
sum(bayes.quantum_times)

In [ ]:
bayes.schedule_pdf

In [ ]:
bayes.pdf_estimation

Finally we can use the evolution of the **QAE** experiments, `schedule_pdf`, and the evolution of the $a$ estimations, `pdf_estimation`, to plot the evolution of the error ($\epsilon$) of the estimation versus the number of calls ($N_A$) to the oracle and compare with MonteCarlo behaviour, $\epsilon = \frac{1}{\sqrt{N_A}}$, and with the Heisenberg limit $\epsilon = \frac{1}{N_A}$

Finally, we can utilize the evolution of the **QAE** experiments (`schedule_pdf`) and the evolution of the $a$ estimations (`pdf_estimation`) to plot the relationship between the estimation error ($\epsilon$) and the number of oracle calls ($N_A$). This allows us to compare the performance of the **BAYESQAE** algorithm with two benchmark behaviors:
- The **Monte Carlo behavior**, where $\epsilon = \frac{1}{\sqrt{N_A}}$.
- The **Heisenberg limit**, where $\epsilon = \frac{1}{N_A}$.

This comparison provides insight into the efficiency and convergence rate of the **BAYESQAE** algorithm relative to classical and quantum limits.

In [ ]:
oracle_calls_ev = (bayes.schedule_pdf["m_k"] * 2 + 1) * bayes.schedule_pdf["shots"].cumsum()
error_ev = np.abs(bayes.pdf_estimation["mean"] - a_good)

plt.plot(
    oracle_calls_ev, 
    error_ev,
    'o'
)

domain_oracle = np.linspace(oracle_calls_ev.min(), oracle_calls_ev.max())

plt.plot(
    domain_oracle, 
    np.sqrt(1.0/domain_oracle),
    '-'
)
plt.plot(
    domain_oracle, 
    1.0/domain_oracle,
    '-'
)

plt.xlabel(r"Oracle Calls ($N_A$)")
plt.ylabel(r"Absolute Error (\epsilon)")

plt.xscale("log")
plt.yscale("log")
plt.legend(["BAYESQAE", r"$\epsilon = \frac{1}{\sqrt{N_{A}}}$", r"$\epsilon = \frac{1}{N_{A}}$"])